![Colegio Bourbaki](./Images/Bourbaki.png)

## Procesamiento de Lenguaje Natural

### Contexto

El objetivo de este notebook es hacer una demostración de la creación de chatbots estilo ChatGPT con conocimiento de datos específicos.

En primer lugar, enseñaremos cómo conectar con el API de Hugging Face para utilizar modelos guardados en ese Hub desde código.

Después, veremos cómo podemos introducir material a la base de conocimiento del chatbot, para así obtener respuestas más personalizadas.

### Librerías

In [1]:
# NLP / LLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    BitsAndBytesConfig
)

# LangChain
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# Utils
import os
from pdfminer.high_level import extract_text
import torch

Primero, definimos el modelo

In [2]:
# Modelo open-weights
model_name = "microsoft/Phi-3.5-mini-instruct"

In [3]:
# Configuración 4-bit para que corra localmente
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# Pipeline de generación "crudo"
generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,
    do_sample=True,
    temperature=0.2,
)

Device set to use cuda:0


In [7]:
prompt = "Explicame en pocas palabras qué es el fine-tuning con LoRA."

In [8]:
outputs = generation_pipeline(prompt)
print(outputs[0]["generated_text"])

Explicame en pocas palabras qué es el fine-tuning con LoRA.

El fine-tuning con LoRA (Low-Rank Adaptation) es una técnica utilizada para ajustar modelos de aprendizaje automático pre-entrenados, especialmente en el contexto de redes neuronales profundas, sin alterar


In [9]:
preguntas = [
    "¿Qué es un modelo de lenguaje grande (LLM)? Respondé en menos de 3 líneas.",
    "¿Qué es RAG (Retrieval-Augmented Generation)? Explicalo con un ejemplo.",
    "¿En qué se diferencia fine-tuning clásico de LoRA?",
]

for i, q in enumerate(preguntas, start=1):
    print(f"\n--- Pregunta {i} ---")
    outputs = generation_pipeline(q)
    print(outputs[0]["generated_text"])


--- Pregunta 1 ---
¿Qué es un modelo de lenguaje grande (LLM)? Respondé en menos de 3 líneas.

Un modelo de lenguaje grande (LLM) es un sistema de inteligencia artificial avanzado entrenado en vastas cantidades de datos textuales para generar, comprender y producir lenguaje humano con un nivel de sofisticación que imita el proces

--- Pregunta 2 ---
¿Qué es RAG (Retrieval-Augmented Generation)? Explicalo con un ejemplo.

# Answer
RAG (Retrieval-Augmented Generation) es una técnica en el campo de la inteligencia artificial y el procesamiento de lenguaje natural que combina la generación de texto con la recuperación de información de una base de datos externa para produ

--- Pregunta 3 ---
¿En qué se diferencia fine-tuning clásico de LoRA?

Fine-tuning clásico y LoRA (Low-Rank Adaptation) son técnicas utilizadas para ajustar modelos pre-entrenados, pero difieren en su enfoque y eficiencia.

1. Fine-tuning clás


Podemos lograr mejores respuestas del modelo si modificamos el atributo **system**.

In [10]:
def build_chat_prompt(system_msg, user_msg):
    return f"<|system|>\n{system_msg}\n<|user|>\n{user_msg}\n<|assistant|>\n"

In [11]:
sistema = "Eres un asistente de poetas, habilidoso en explicar conceptos complejos de programación creativamente."
usuario = "Compón un poema que explique el concepto de recursión en programación."

In [12]:
prompt = build_chat_prompt(sistema, usuario)

In [13]:
out = generation_pipeline(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
)

print(out[0]["generated_text"])

<|system|>
Eres un asistente de poetas, habilidoso en explicar conceptos complejos de programación creativamente.
<|user|>
Compón un poema que explique el concepto de recursión en programación.
<|assistant|>
 En las profundidades de la programación, donde los algoritmos se encuentran,
Existe un concepto, un giro, donde la intuición se vuelve.
Recursión, el término, que suena y recuerda,
En líneas de código, una historia que escribe.

Una función en llamada, a sí misma, se dirige,
Mientras se avanza hacia el reino del reto.
En la esencia misma, encuentra su propio hilo,
Envolviéndose en sí, en un atajo elegante.

Un problema, una vez grande, como un monte extenso,
Se divide en partes, para su resolución enmendarse.
A través de sí misma, se desplaza, sin temor ni desesperación,
En cada nivel, una parte, un pequeño paso.

Una recursión, como una torre, se construye,
Por cada capa que añade, un nodo más.
Cada llamada, una capa, un paso más allá,
En la cima, el problema


### Creación de un asistente especializado

A su vez, podemos aprovechar aún más las capacidades de loss LLM haciendo una especie de fine-tuning. La idea consiste en alimentar al modelos con documentos propios para así lograr respuestas informadas sobre ellos.

Esto es posible a través de los encajes y la generación de una base de datos vectorizada.

Primero, extraemos texto desde archivos pdfs...


... y extramos el texto de un pdf para después guardarlo en .txt.

In [14]:
pdf_path = "/media/pdconte/hdd/Pablo/Personal/Colegio_Bourbaki/Natural_Language_Processsing/Semana5/Data/Feynman1982_Article_SimulatingPhysicsWithComputers.pdf"
output_txt_path = "./Data/output.txt"

# Extraer texto del PDF a un archivo de texto
text = extract_text(pdf_path)

os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)
with open(output_txt_path, "w", encoding="utf-8") as f:
    f.write(text)

print("Texto extraído y guardado en", output_txt_path)

Texto extraído y guardado en ./Data/output.txt


In [15]:
loader = TextLoader("./Data/output.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100, separator="\n"
)

texts = text_splitter.split_documents(documents)

In [16]:
texts[0]

Document(metadata={'source': './Data/output.txt'}, page_content="International Journal of Theoretical Physics, VoL 21, Nos. 6/7,  1982 \nSimulating Physics with Computers \nRichard P. Feynman \nDepartment of Physics, California Institute of Technology, Pasadena, California 91107 \nReceived May 7, 1981 \n1.  I N T R O D U C T I O N  \nOn  the  program  it  says  this  is  a  keynote  speech--and  I  don't  know \nwhat  a  keynote speech is.  I  do not intend  in  any way to suggest what  should \nbe  in  this  meeting  as  a  keynote of  the  subjects  or  anything  like  that.  I  have \nmy  own  things  to  say  and  to  talk  about  and  there's  no  implication  that \nanybody  needs  to  talk  about  the  same  thing  or  anything  like  it.  So  what  I \nwant  to  talk  about  is  what  Mike  Dertouzos  suggested  that  nobody  would \ntalk  about.  I  want  to  talk  about  the  problem  of  simulating  physics  with \ncomputers  and  I  mean  that  in  a  specific way  which  I

In [17]:
len(texts)

72

En seguida, cargamos el documento a la base de datos.

In [18]:
# Embeddings HuggingFace (sin costo de API)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Crear / persistir base vectorial con Chroma
db = Chroma.from_documents(texts, embeddings, collection_name="docs_rag")

Este modelo ahora se puede utilizar como un ChatGPT con conocimiento especializado:

In [19]:
torch.cuda.empty_cache()

In [20]:
generation_pipeline_db = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.2,
    return_full_text=False,
)

local_llm = HuggingFacePipeline(pipeline=generation_pipeline_db)

Device set to use cuda:0


In [21]:
retriever = db.as_retriever(search_kwargs={"k": 1})
qa = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="stuff",
    retriever=retriever,
)

# Ejemplo de consulta
query = "¿De qué trata el documento? Resume los puntos principales."
result = qa.invoke({"query": query})

In [22]:
respuesta = result["result"]
respuesta

'\nEl documento trata sobre una conferencia titulada "Simulando Física con Computadores" presentada por Richard P. Feynman. Feynman explica que su discurso no tiene la intención de establecer un tema o keynote para el evento, sino que pretende abordar el tema sugerido por Mike Dertouzos, que no está especificado en el texto proporcionado. Los puntos principales incluyen la clarificación de que el discurso de Feynman no es un keynote, y su intención es disc'

In [23]:
query = "How photons are polarized?"
result = qa.invoke({"query": query})
result['result']

' Photons are polarized such that they either go into the ordinary ray or the extraordinary one, but not both. The probability of a photon being in either ray is always 100%, meaning a photon will always be found in one or the other, never both.\n\nContext: places  out,  where  the  photon  can  go.  (See  Figure  2.) \nIf you put  a  polarized photon  in,  then  it will  go  to  one beam called the \nordinary  ray, or  another,  the  extraordinary'

In [24]:
query = "Expand the concept of negative probabilities"
result = qa.invoke({"query": query})
result["result"]

'\n\nNegative probabilities are not a concept that exists in the real world or in standard probability theory. Probabilities, by definition, are non-negative values that represent the likelihood of an event occurring, ranging from 0 (the event will not occur) to 1 (the event will certainly occur). \n\nIn the context of quantum mechanics, the "negative probabilities" you\'re referring to might be a misunderstanding or misrepresentation of the complex probability amplitudes. In quantum mechanics, probabilities are derived from the square of the magnitude of a complex number (probability'

In [25]:
query = "What is a quantum computer?"
result = qa.invoke({"query": query})
result["result"]

' A quantum computer is a type of computing device that uses the principles of quantum mechanics to process information. Unlike classical computers, which use bits as the basic unit of information, quantum computers use quantum bits, or qubits, which can exist in multiple states simultaneously due to the phenomenon of superposition. This allows quantum computers to potentially solve complex problems much faster than classical computers.\n\n\nThe context provided discusses the author\'s struggle to understand quantum mechanics and the concept of a "world view" associated with it. The author expresses difficulty in grasping the principles of quantum mechanics, and even though they suspect'

Podemos comparar la salida con el modelo original:

In [26]:
out = generation_pipeline(
    query,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
)

print(out[0]["generated_text"])

What is a quantum computer? A quantum computer is a type of computing device that uses the principles of quantum mechanics to perform calculations. It operates on quantum bits, or qubits, which can exist in multiple states simultaneously due to the concept of superposition. This allows quantum computers to process a vast number of possibilities simultaneously, potentially solving complex problems much faster than classical computers.

In contrast to classical bits, which can either be a 0 or a 1, qubits can represent a 0, a 1, or any quantum superposition of these states. This ability, combined with another quantum property called entanglement, where the state of


### Preguntas

**Uso de LLM:**

4) ¿Qué ventajas ofrece usar un LLM para la creación de chatbots comparado con versiones anteriores?
5) ¿Cómo se formulan las peticiones al modelo para generar respuestas coherentes y relevantes?
6) ¿Qué limitaciones tienes estos modelos y cómo puedes mitigarlas?

**Introducción de Material a la Base de Conocimiento:**

7) ¿Cómo puedes personalizar las respuestas del modelo utilizando información específica?
8) ¿Cuál es la importancia de la relevancia y precisión del material que se introduce en la base de conocimientos del bot?
9) ¿Qué estrategias se pueden utilizar para mantener actualizada la base de conocimientos del chatbot?

**Personalización y Respuestas del Chatbot:**

10) ¿De qué manera se puede ajustar el tono o el estilo de las respuestas que genera el modelo?
11) ¿Cómo afecta el contexto proporcionado a las respuestas generadas por el chatbot?
    Describe un método para evaluar la precisión y utilidad de las respuestas del chatbot.

**Problemas Éticos y de Privacidad:**

12) ¿Cuáles son las consideraciones éticas al utilizar modelos de lenguaje generativos en un chatbot?
13) ¿Cómo debería manejar un chatbot las solicitudes de datos personales o sensibles de los usuarios?
14) ¿Qué medidas se pueden tomar para garantizar la privacidad y la seguridad de los usuarios al interactuar con un chatbot?

![Lenguaje Matemático](./Images/Matematicas.png)